In [3]:
%pip install newspaper3k


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 2.5 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 1.0 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.0/186.0 kB 906.3 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 1.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 1.5 MB/s eta 0:00:0000:0100:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 2.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 1.5 MB/s eta 0:00:00 0:00:01
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13541 sha256=befa01b5191fa21674fa0db1c915740f9189b2b56ab6c8c473a190ce9cf5d51d
  Stored in directory: /Users/fenilvadher/Library/Caches/

In [5]:
%pip install lxml_html_clean



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install tf-keras


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 1.2 MB/s eta 0:00:0000:0100:010m

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
# utils.py

from serpapi import GoogleSearch
from newspaper import Article
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline
import pandas as pd
from datetime import datetime

In [9]:
SERP_API_KEY = "88e9e548ed93ca0317faeef96410bf0a"  # Replace with your SerpAPI key
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Error while downloading from https://cdn-lfs.hf.co/sshleifer/distilbart-cnn-12-6/3bac65d18c99463302d12ca75c2220ea714f9c81ce235f205fa818efe71df6ea?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27pytorch_model.bin%3B+filename%3D%22pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1744627658&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NDYyNzY1OH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9zc2hsZWlmZXIvZGlzdGlsYmFydC1jbm4tMTItNi8zYmFjNjVkMThjOTk0NjMzMDJkMTJjYTc1YzIyMjBlYTcxNGY5YzgxY2UyMzVmMjA1ZmE4MThlZmU3MWRmNmVhP3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9zaXRpb249KiZyZXNwb25zZS1jb250ZW50LXR5cGU9KiJ9XX0_&Signature=EKTXAhrhl34Myos2mnnr96xiPOtetsWVIQOKF2jVFJgUUBurh6tOHAlLPx-WSQyvMLt

In [10]:
def search_google(query, num_results=3):
    params = {
        "engine": "google",
        "q": query,
        "api_key": SERP_API_KEY
    }
    search = GoogleSearch(params)
    results = search.get_dict()
    urls = []

    if "organic_results" in results:
        for result in results["organic_results"][:num_results]:
            if "link" in result:
                urls.append(result["link"])
    
    return urls

In [11]:
def extract_article(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        return article.text
    except Exception as e:
        print(f"Error extracting from {url}: {e}")
        return None

In [12]:
def summarize_text(text, max_tokens=512):
    if len(text.split()) > 100:
        try:
            summary = summarizer(text[:1000], max_length=120, min_length=30, do_sample=False)
            return summary[0]['summary_text']
        except:
            return "Summary not available."
    return "Text too short to summarize."

In [13]:
def check_similarity(user_input, articles):
    user_embedding = model.encode(user_input, convert_to_tensor=True)
    best_score = 0
    best_summary = None

    for article in articles:
        if article:
            article_embedding = model.encode(article, convert_to_tensor=True)
            score = util.pytorch_cos_sim(user_embedding, article_embedding).item()
            if score > best_score:
                best_score = score
                best_summary = summarize_text(article)

    is_real = best_score >= 0.7
    return best_score, is_real, best_summary

In [14]:
def log_result(news_input, result, score):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log_data = pd.DataFrame([{
        "timestamp": timestamp,
        "news_input": news_input,
        "result": result,
        "score": round(score, 3)
    }])
    log_data.to_csv("logs.csv", mode='a', header=not pd.io.common.file_exists("logs.csv"), index=False)